In [1]:
## Imports
## imports
import os
import tensorflow as tf
import numpy as np

import models.component as cP
import models.data_process as dP
import math

from multiprocessing import Process

# Plot Data
import matplotlib.pyplot as plt



In [2]:
# Global
EPOCH = 100
fps_size = 2048

In [3]:
def train_class1():
    
    fps_dict = dP.process_data_np(ds_name = 'LD50_Test_data', pool_size = 4)

    fps1_train = fps_dict['fps1_train_np']
    fps1_test = fps_dict['fps1_test_np']
    labels1_train = fps_dict['labels1_train_np']
    labels1_test = fps_dict['labels1_test_np']
    i_lr = 0.005
    lastLoss = 0
    layers_dim = np.array([256, 64, 10], dtype = np.float32)
    fps_dim = 2048
    batch_size = 64
    with tf.device('/gpu:1'):

        labels = tf.placeholder(tf.float32, shape=[None, 10], name = 'labels')
        is_train_enc = tf.placeholder(tf.bool, name = 'is_train_enc')
        fps = tf.placeholder(tf.float32, shape = [None, 2048,2], name ='fps')
        lr_1 = tf.placeholder(tf.float32, name = 'lr_1')
        
        predict, _ = cP.cnn_encoder(fps, fps_dim, layers_dim, is_train_enc, reuse=False)
        classifyer1_loss = tf.losses.softmax_cross_entropy(labels,predict)
        
        predicted_labels = tf.one_hot(tf.math.argmax(predict, axis = 1), 10, on_value = 1.0, off_value = 0.0)
        classifyer1_acc = tf.metrics.accuracy(labels,predicted_labels)
        
        t_vars = tf.trainable_variables()
        enc_vars = [var for var in t_vars if 'encode' in var.name]
        
        trainer_enc = tf.train.AdamOptimizer(learning_rate =lr_1, beta1=0.9, beta2=0.999,
                                             epsilon=1e-08, use_locking=False,
                                             name='Adam_encoder').minimize(classifyer1_loss, var_list=enc_vars)
        

        
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    num_batch_epoch = math.floor(len(fps1_train)/batch_size)

    for i in range(EPOCH):
        batch1_train = dP.batch_gen(fps1_train, labels1_train, batch_size = batch_size, FLAG = True)
        batch1_val = dP.batch_gen(fps1_test, labels1_test, batch_size = batch_size, FLAG = True) 

        for j in range(num_batch_epoch):

            dict_train =  {fps: batch1_train['fps'][j], labels: batch1_train['label'][j], is_train_enc: True, lr_1: i_lr}
            sess.run([trainer_enc], feed_dict = dict_train)

        dict_train_loss = {fps: fps1_train, labels: labels1_train, is_train_enc: False}
        trainLoss, trainAcc = sess.run([classifyer1_loss, classifyer1_acc], feed_dict = dict_train_loss)
        if i%5 == 0 and i != 0:
            if lastLoss - trainLoss <= 0.00000:
                i_lr = max([i_lr * 0.5, 0.00001])
            lastLoss = trainLoss 
        dict_val =  {fps: fps1_test, labels: labels1_test, is_train_enc: False}
        valLoss, valAcc = sess.run([classifyer1_loss, classifyer1_acc], feed_dict = dict_val)
        #encoder_dict={fps: fps_test, is_train_enc: False}
        #print(sess.run(predict, feed_dict = encoder_dict))
        
        print('1 %f trainLoss = %f valLoss = %f trainAcc = %f valAcc = %f' % (i_lr, trainLoss, valLoss, trainAcc[0], valAcc[0]))
    sess.close()

def train_class2():
    fps_dict = dP.process_data_np(ds_name = 'LD50_Test_data', pool_size = 4)

    fps_train = fps_dict['fps2_train_np']
    fps_test = fps_dict['fps2_test_np']
    labels_train = fps_dict['labels2_train_np']
    labels_test = fps_dict['labels2_test_np']
    i_lr = 0.01
    lastLoss = 0
    layers_dim = np.array([256, 64, 10], dtype = np.float32)
    fps_dim = 512
    batch_size = 64

        
    with tf.device('/gpu:0'):
        labels2 = tf.placeholder(tf.float32, shape=[None, 10], name = 'labels2')
        is_train_dense = tf.placeholder(tf.bool, name = 'is_train_dense')
        fps2 = tf.placeholder(tf.float32, shape = [None, 512], name ='fps2')
        lr_2 = tf.placeholder(tf.float32, name = 'lr_2')
        predict2 = cP.dense_encoder(fps2, 512, layers_dim, is_train_dense, reuse=False)
        classifyer2_loss = tf.losses.softmax_cross_entropy(labels2,predict2)
        predicted_labels = tf.one_hot(tf.math.argmax(predict2, axis = 1), 10, on_value = 1.0, off_value = 0.0)
        classifyer2_acc = tf.metrics.accuracy(labels2,predicted_labels)
        
        t_vars = tf.trainable_variables()
        dense_vars = [var for var in t_vars if 'dense_class' in var.name]
        trainer_dense = tf.train.AdamOptimizer(learning_rate =lr_2, beta1=0.9, beta2=0.999,
                                             epsilon=1e-08, use_locking=False,
                                             name='Adam_encoder').minimize(classifyer2_loss, var_list=dense_vars)
        
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    num_batch_epoch = math.floor(len(fps_train)/batch_size)

    for i in range(EPOCH):
        batch1_train = dP.batch_gen(fps_train, labels_train, batch_size = batch_size, FLAG = True)
        batch1_val = dP.batch_gen(fps_test, labels_test, batch_size = batch_size, FLAG = True) 

        for j in range(num_batch_epoch):

            dict_train =  {fps2: batch1_train['fps'][j], labels2: batch1_train['label'][j], is_train_dense: True, lr_2: i_lr}
            sess.run([trainer_dense], feed_dict = dict_train)

        dict_train_loss = {fps2: fps_train, labels2: labels_train, is_train_dense: False}
        trainLoss, trainAcc = sess.run([classifyer2_loss, classifyer2_acc], feed_dict = dict_train_loss)
        if i%5 == 0 and i != 0:
            if lastLoss - trainLoss <= 0.00000:
                i_lr = max([i_lr * 0.5, 0.00001])
            lastLoss = trainLoss 
        dict_val =  {fps2: fps_test, labels2: labels_test, is_train_dense: False}
        valLoss, valAcc = sess.run([classifyer2_loss, classifyer2_acc], feed_dict = dict_val)
        #encoder_dict={fps: fps_test, is_train_enc: False}
        #print(sess.run(predict, feed_dict = encoder_dict))
        
        print('2 %f trainLoss = %f valLoss = %f trainAcc = %f valAcc = %f' % (i_lr, trainLoss, valLoss, trainAcc[0], valAcc[0]))
    sess.close()

In [4]:
## Main

if __name__ == "__main__":
    
    Process(target=train_class1).start()
    Process(target=train_class2).start()
  
    

2 0.010000 trainLoss = 1.364407 valLoss = 1.425479 trainAcc = 0.000000 valAcc = 0.919290
2 0.010000 trainLoss = 1.185427 valLoss = 1.382215 trainAcc = 0.918512 valAcc = 0.923164
2 0.010000 trainLoss = 1.108067 valLoss = 1.373816 trainAcc = 0.922472 valAcc = 0.925707
2 0.010000 trainLoss = 1.022283 valLoss = 1.333350 trainAcc = 0.925253 valAcc = 0.927469
2 0.010000 trainLoss = 0.970886 valLoss = 1.301398 trainAcc = 0.927227 valAcc = 0.929114
2 0.005000 trainLoss = 0.899935 valLoss = 1.284559 trainAcc = 0.928811 valAcc = 0.930718
1 0.005000 trainLoss = 2.097891 valLoss = 2.094016 trainAcc = 0.000000 valAcc = 0.843816
2 0.005000 trainLoss = 0.808375 valLoss = 1.279806 trainAcc = 0.930449 valAcc = 0.932722
2 0.005000 trainLoss = 0.788193 valLoss = 1.293470 trainAcc = 0.932413 valAcc = 0.934406
2 0.005000 trainLoss = 0.768822 valLoss = 1.325312 trainAcc = 0.934064 valAcc = 0.935899
1 0.005000 trainLoss = 1.847861 valLoss = 1.830327 trainAcc = 0.844463 valAcc = 0.869694
2 0.005000 trainLoss 

2 0.000625 trainLoss = 0.319466 valLoss = 1.335602 trainAcc = 0.964382 valAcc = 0.964630
2 0.000625 trainLoss = 0.311503 valLoss = 1.336716 trainAcc = 0.964521 valAcc = 0.964763
1 0.002500 trainLoss = 1.689000 valLoss = 1.630874 trainAcc = 0.899382 valAcc = 0.899455
2 0.000625 trainLoss = 0.309724 valLoss = 1.337143 trainAcc = 0.964653 valAcc = 0.964892
2 0.000625 trainLoss = 0.315963 valLoss = 1.341840 trainAcc = 0.964784 valAcc = 0.965018
1 0.002500 trainLoss = 1.667472 valLoss = 1.638782 trainAcc = 0.899499 valAcc = 0.899557
2 0.000625 trainLoss = 0.311016 valLoss = 1.336087 trainAcc = 0.964910 valAcc = 0.965141
2 0.000625 trainLoss = 0.309111 valLoss = 1.344336 trainAcc = 0.965037 valAcc = 0.965262
2 0.000625 trainLoss = 0.323108 valLoss = 1.334678 trainAcc = 0.965158 valAcc = 0.965380
2 0.000625 trainLoss = 0.314447 valLoss = 1.335642 trainAcc = 0.965275 valAcc = 0.965493
1 0.002500 trainLoss = 1.664930 valLoss = 1.638414 trainAcc = 0.899603 valAcc = 0.899655
2 0.000625 trainLoss 

1 0.001250 trainLoss = 0.886042 valLoss = 1.831745 trainAcc = 0.912462 valAcc = 0.912762
1 0.001250 trainLoss = 0.846883 valLoss = 1.839691 trainAcc = 0.912721 valAcc = 0.913046
1 0.001250 trainLoss = 0.814953 valLoss = 1.851719 trainAcc = 0.913000 valAcc = 0.913335
1 0.001250 trainLoss = 0.909949 valLoss = 1.877985 trainAcc = 0.913293 valAcc = 0.913560
1 0.001250 trainLoss = 0.838479 valLoss = 1.863627 trainAcc = 0.913514 valAcc = 0.913812
1 0.001250 trainLoss = 0.812110 valLoss = 1.879627 trainAcc = 0.913762 valAcc = 0.914083
1 0.001250 trainLoss = 0.778242 valLoss = 1.863938 trainAcc = 0.914029 valAcc = 0.914362
1 0.001250 trainLoss = 0.788232 valLoss = 1.861355 trainAcc = 0.914314 valAcc = 0.914641
1 0.001250 trainLoss = 0.783237 valLoss = 1.862386 trainAcc = 0.914595 valAcc = 0.914915
1 0.001250 trainLoss = 0.795431 valLoss = 1.863848 trainAcc = 0.914862 valAcc = 0.915171
1 0.001250 trainLoss = 0.756165 valLoss = 1.870702 trainAcc = 0.915118 valAcc = 0.915445
1 0.001250 trainLoss 